# Poincare Sphere Visualizations

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

# Effective medium model imports
from effmed.lib.matrix_model import effective_medium
from effmed.lib.supplemental import dB, rotational_transform, coherence

In [ ]:
# Define a function to get vectors 

def get_poincare_sphere(em):
    
    # Component wave amplitudes
    E0x = abs(em.HH)
    E0y = abs(em.HV)
    # Component phase
    ϕHH = np.angle(em.HH)
    ϕHV = np.angle(em.HV)
    
    # ellipse phase
    ϕ = ϕHH-ϕHV
    ϕ[ϕ<-np.pi] += 2.*np.pi
    ϕ[ϕ>np.pi] -= 2.*np.pi

    # get vectors from wave amplitude and ellipse phase
    S1 = E0x**2. - E0y**2.
    S2 = 2.*E0x*E0y*np.cos(ϕ)
    S3 = 2.*E0x*E0y*np.sin(ϕ)
    
    return S1,S2,S3

In [ ]:
# Initialize the model at a chosen center frequency
em = effective_medium()
em.system_setup(fc=300e6)

# Set the material properties
lams = [0.,.1,0.9]
em.ice_properties(T=253.,epsr=3.15,lam=lams)

# Set the domain
dphidz = (4.*np.pi*em.fc/em.c)*(em.depsr*(lams[1]-lams[0])/(2.*np.sqrt(em.epsr)))
H = 2.*np.pi/dphidz
zs = np.arange(.1,H)

# Solve model for all layers
em.solve(zs,H,0.,0.001,lams)

# Rotate model result to all azimuths for a 2-d image
rotational_transform(em,n_thetas=401)

# Calculate HH-VV phase coherence
coherence(em)

# Save the model class to distinguish between others
em1 = em

In [ ]:
# Initialize the model at a chosen center frequency
em = effective_medium()
em.system_setup(fc=300e6)

# Set the material properties
em.ice_properties(T=253.,epsr=3.15,lam=[.333,.333,.333])

# Set the domain
H = 1000
zs = np.arange(10,H)
gammas = np.array([1,.2])

# Solve model for all layers
em.solve(zs,H,0.,0.0,lams,gammas=gammas)

# Rotate model result to all azimuths for a 2-d image
rotational_transform(em,n_thetas=401)

# Calculate HH-VV phase coherence
coherence(em)

em2 = em

In [ ]:
# Initialize the model at a chosen center frequency
em = effective_medium()
em.system_setup(fc=300e6)

# Set the material properties
lams = [.15,.35,0.5]
em.ice_properties(T=253.,epsr=3.15,lam=lams)

# Set the domain
dphidz = (4.*np.pi*em.fc/em.c)*(em.depsr*(lams[1]-lams[0])/(2.*np.sqrt(em.epsr)))
H = 2.*np.pi/dphidz
zs = np.arange(.1,H)
gammas = np.array([1.,.2])

# Solve model for all layers
em.solve(zs,H,0.,0.,lams,gammas=gammas)

# Rotate model result to all azimuths for a 2-d image
rotational_transform(em,n_thetas=401)

# Calculate HH-VV phase coherence
coherence(em)

em3 = em

In [ ]:
# Plot each of the model results on the Poincare Sphere

plt.figure(figsize=(12,4))
ax1 = plt.subplot(131,projection='3d')
ax1.axis('off')
ax2 = plt.subplot(132,projection='3d')
ax2.axis('off')
ax3 = plt.subplot(133,projection='3d')
ax3.axis('off')

# Create a meshgrid for reference on the Poincare Sphere
u, v = np.mgrid[0:2*np.pi+.01:np.pi/10, 0:np.pi:25j]
x = np.cos(u)*np.sin(v)
y = np.sin(u)*np.sin(v)
z = np.cos(v)
# Plot the grid in all three subplots
ax1.plot_wireframe(y, z, x, color="k", alpha=0.025)
ax2.plot_wireframe(y, z, x, color="k", alpha=0.025)
ax3.plot_wireframe(y, z, x, color="k", alpha=0.025)

# For all three model results (one in each subplot)
ems = [em1,em2,em2]
axs = [ax1,ax2,ax3]
S1, S2, S3 = get_poincare_sphere(em1)
for i in range(0,len(S1[0])//2+2,20):
    ax1.plot(S2[:,i],-S1[:,i],S3[:,i],lw=2,c=cm.twilight_shifted(i/200))
S1, S2, S3 = get_poincare_sphere(em2)
for i in range(0,len(S1[0]),20):
    ax2.plot(S2[0,i],-S1[0,i],S3[0,i],'.',ms=3.5,c=cm.twilight_shifted(i/400))
S1, S2, S3 = get_poincare_sphere(em3)
for i in range(len(S1[0])-1,-1,-20):
    ax3.plot(S2[:,i],-S1[:,i],S3[:,i],lw=2,c=cm.twilight_shifted(i/400))
ax3.plot(S2[0,400],-S1[0,400],S3[0,400],'.',c=cm.twilight_shifted(.5))
ax3.plot(S2[0,0],-S1[0,0],S3[0,0],'.',c=cm.twilight_shifted(1.))

ax1.axis('equal')
ax2.axis('equal')
ax3.axis('equal')

In [ ]:
# Model the case with a COF rotation at depth

# Initialize the model at a chosen center frequency
em = effective_medium()
em.system_setup(fc=300e6)

# Set the material properties
lams = [.15,.35,0.5]
em.ice_properties(T=253.,epsr=3.15,lam=lams)

# Set the domain
dphidz = (4.*np.pi*em.fc/em.c)*(em.depsr*(lams[1]-lams[0])/(2.*np.sqrt(em.epsr)))
H = 8.*np.pi/dphidz
layer_dz = 2.
zs = np.arange(1,H,layer_dz)
dzs = layer_dz*np.ones(len(zs))
thetas = np.zeros(len(zs))
lams = np.tile(lams,(len(zs),1))

# Azimuthal rotation of the COF at depth 200 m
psis = .01+np.zeros_like(zs)
psis[zs>350] = .6

# Solve model for all layers
em.solve(zs,dzs,thetas,psis,lams)

# Rotate model result to all azimuths for a 2-d image
rotational_transform(em,n_thetas=401)

# Calculate HH-VV phase coherence
coherence(em)

In [ ]:
# Plot the rotated model scenario on the Poincare Sphere

plt.figure(figsize=(12,4))
ax1 = plt.subplot(131,projection='3d')
ax1.axis('off')
ax2 = plt.subplot(132,projection='3d')
ax2.axis('off')
ax3 = plt.subplot(133,projection='3d')
ax3.axis('off')

ax1.plot_wireframe(y, z, x, color="k", alpha=0.025)
ax2.plot_wireframe(y, z, x, color="k", alpha=0.025)
ax3.plot_wireframe(y, z, x, color="k", alpha=0.025)

S1, S2, S3 = get_poincare_sphere(em)
di = 175
ai = 70
ax1.plot(S2[:di,ai],-S1[:di,ai],S3[:di,ai],lw=2,c=cm.twilight_shifted(ai/200))
ax1.plot(S2[di:,ai],-S1[di:,ai],S3[di:,ai],lw=2,c='grey')
ax1.plot(S2[di,ai],-S1[di,ai],S3[di,ai],'.',ms=10,mew=2,mfc='w',c='grey')
ai = 110
ax2.plot(S2[:di,ai],-S1[:di,ai],S3[:di,ai],lw=2,c=cm.twilight_shifted(ai/200))
ax2.plot(S2[di:,ai],-S1[di:,ai],S3[di:,ai],lw=2,c='grey')
ax2.plot(S2[di,ai],-S1[di,ai],S3[di,ai],'.',ms=10,mew=2,mfc='w',c='grey')
ai = 130
ax3.plot(S2[:di,ai],-S1[:di,ai],S3[:di,ai],lw=2,c=cm.twilight_shifted(ai/200))
ax3.plot(S2[di:,ai],-S1[di:,ai],S3[di:,ai],lw=2,c='grey')
ax3.plot(S2[di,ai],-S1[di,ai],S3[di,ai],'.',ms=10,mew=2,mfc='w',c='grey')

ax1.axis('equal')
ax2.axis('equal')
ax3.axis('equal')